In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import load_dataset
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer, BertForSequenceClassification, BertModel, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl
import torchmetrics
from sklearn.model_selection import train_test_split
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

df = pd.read_csv('../data/hate_speech_preprocessed.csv')

c:\Users\mariu\anaconda3\envs\comp0197-pt\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
l = [0,1,1,1]
l[int(df['label'].iloc[0])]


0

In [22]:
def preprocessing(df):     
    # 0: comment is not hateful
    # 1: comment is hateful (target_race)
    # 2: comment is hateful (target_religion)
    # 3: comment is hateful (target_origin)
    # 4: comment is hateful (target_gender)
    # 5: comment is hateful (target_sexuality)
    # 6: comment is hateful (target_age)
    # 7: comment is hateful (target_disability)
    l = []
    for i in range(len(df)):
        list_class = [0] * 8
        list_class[int(df['label'].iloc[i])] = 1
        l.append(list_class)
    
    df['Class'] = l
    
    return df

def create_class(i):
    """
    Creating a list of lenght 8 with only zeros except from a one at index i
    """
    l = [0] * 8
    l[i]=1
    return l
    
df = preprocessing(df)

,comment_id,text,hate_speech_score,label,Class
0,1,White people are trash.,0.46,0,"[1, 0, 0, 0, 0, 0, 0, 0]"
1,2,"You've caught the big gay, it's known to be ai...",0.03,0,"[1, 0, 0, 0, 0, 0, 0, 0]"
2,3,"I'm not saying you said that, I'm just saying ...",-1.29,0,"[1, 0, 0, 0, 0, 0, 0, 0]"
3,4,Donald Trump. Yeet myself off a building onto ...,-0.24,0,"[1, 0, 0, 0, 0, 0, 0, 0]"
4,5,Fabrice Fabrice is ostensibly black or black/l...,-2.84,0,"[1, 0, 0, 0, 0, 0, 0, 0]"


In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [3]:
model = model.to('cuda')

In [38]:
df['text'][:10]

0                              White people are trash.
1    You've caught the big gay, it's known to be ai...
2    I'm not saying you said that, I'm just saying ...
3    Donald Trump. Yeet myself off a building onto ...
4    Fabrice Fabrice is ostensibly black or black/l...
5    First off you look cool as fuck! Anyway if we ...
6    \*points to posters asking for palestinian rig...
7    Guranteed at least one of these guys has raped...
8    They'll come back in your plan, also. Plus we ...
9                                   eat my fuck, bitch
Name: text, dtype: object

In [4]:
MAXIMUM = 1000

X = list(df['text'][:MAXIMUM])
y = list(df['label'][:MAXIMUM])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train)

In [5]:
X_train_tokenize = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenize = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [6]:
class HateSpeechData(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __getitem__(self, index):
        item = {key:torch.tensor(value[index]) for key, value in self.X.items()}
        item["labels"] = torch.tensor(self.y[index])
        return item
    
    def __len__(self):
        return len(self.X["input_ids"])

In [ ]:
class BERTForFineTuning(torch.nn.Module):
    def __init__(self):
        super(BERTForFineTuning, self).__init__()
        # first layer is the bert
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        # apply a dropout
        self.l2 = torch.nn.Dropout(0.3)
        # feature bert input is 768 and we want the prediction on the 8 class
        self.l3 = torch.nn.Linear(768, 8)
    
    def forward(self, ids, mask, token_type_ids):

        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output
    

def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

def train(nm_epoch, training_loader):
    
    model = BERTForFineTuning()
    model.to("cuda")
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-5)

    # set the model to training mode
    model.train()
    for epoch in nm_epoch:
        running_loss = 0.0
        for i, data in enumerate(training_loader, 0):
            ids = data['input_ids']
            attention_mask = data['attention_mask']
            token_type_ids = data['token_type_ids']
            labels = data['labels']

            # initialize the optimizer
            optimizer.zero_grad()
            #forward inputs
            outputs = model.forward(ids, attention_mask, token_type_ids)
            # define the loss
            loss = loss_fn(outputs, labels)
            # backpropagate
            loss.backward()
            optimizer.step()
            # add the loss to the running loss
            running_loss+=loss.item()

            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
    
    try:
        torch.save(model.state_dict(), 'fine_tuned_bert.pt')
        print('Model has been saved !')
    except:
        print('The model has already been saved!')

In [7]:
train_dataset = HateSpeechData(X_train_tokenize, y_train)
val_dataset = HateSpeechData(X_val_tokenize, y_val)

In [8]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

def compute_metrics(p):
    pred, true_y = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=true_y, y_pred=pred)
    recall = recall_score(y_true=true_y, y_pred=pred)
    f1= f1_score(y_true=true_y, y_pred=pred)
    precision= precision_score(y_true=true_y, y_pred=pred)

    return {'accuracy' :accuracy, 'recall': recall, 'f1': f1, 'precision': precision }

In [9]:
arguments = TrainingArguments(
    output_dir = "output", 
    num_train_epochs=1,
    per_device_train_batch_size=8
)

trainer = Trainer(
    model=model, 
    args=arguments, 
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

<h1>Mulitclass classification fine tuned bert</h1>

In [77]:
# libraries

import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import load_dataset
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer, BertForSequenceClassification, BertModel, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl
import torchmetrics
from sklearn.model_selection import train_test_split

# import + preprocess the data
def preprocessing(df):     
    # 0: comment is not hateful
    # 1: comment is hateful (target_race)
    # 2: comment is hateful (target_religion)
    # 3: comment is hateful (target_origin)
    # 4: comment is hateful (target_gender)
    # 5: comment is hateful (target_sexuality)
    # 6: comment is hateful (target_age)
    # 7: comment is hateful (target_disability)
    l = []
    for i in range(len(df)):
        list_class = [0] * 8
        list_class[int(df['label'].iloc[i])] = 1
        l.append(list_class)
    
    df['Class'] = l
    
    return df

def create_class(i):
    """
    Creating a list of lenght 8 with only zeros except from a one at index i
    """
    l = [0] * 8
    l[i]=1
    return l



# Custome the data for our need
class HateSpeechData(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __getitem__(self, index):
        item = {key:torch.tensor(value[index]) for key, value in self.X.items()}
        item["labels"] = torch.tensor(self.y[index], dtype=torch.float)
        return item
    
    def __len__(self):
        return len(self.X["input_ids"])
    

# Dataloader
def dataloader(df, val_frac, test_frac, batch_size, max_lenght=None):

    if max_lenght is None:
        X = list(df['text'])
        y = list(df['Class'])
    else:
        X = list(df['text'][:MAXIMUM])
        y = list(df['Class'][:MAXIMUM])

    # split the data
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_frac, stratify=y)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=test_frac, stratify=y_train)

    # initialize the tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Tokenize the comment text --> create an object that has free keys : input_ids, attention_mask, 
    X_train_tokenize = tokenizer(X_train, padding=True, truncation=True, max_length=512)
    X_val_tokenize = tokenizer(X_val, padding=True, truncation=True, max_length=512)  
    X_test_tokenize = tokenizer(X_test, padding=True, truncation=True, max_length=512)
    
    train_dataset = HateSpeechData(X_train_tokenize, y_train)
    val_dataset = HateSpeechData(X_val_tokenize, y_val)
    test_dataset = HateSpeechData(X_test_tokenize, y_test)

    trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    validloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle = True, num_workers=0)

    return trainloader, validloader, testloader

    
#Create the BERT model we will use to fine tuned
class BERTForFineTuning(torch.nn.Module):
    def __init__(self):
        super(BERTForFineTuning, self).__init__()
        # first layer is the bert
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        # apply a dropout
        self.l2 = torch.nn.Dropout(0.3)
        # feature bert input is 768 and we want the prediction on the 8 class
        self.l3 = torch.nn.Linear(768, 8)
    
    def forward(self, ids, mask, token_type_ids):

        output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1[1])
        output = self.l3(output_2)
        return output
    

def loss_fn(outputs, targets):
    print(len(targets))
    print(len(outputs))
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

def train(nm_epoch, training_loader):
    
    model = BERTForFineTuning()
    # device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to('cpu')
    optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-5,capturable=True )

    # set the model to training mode
    model.train()
    for epoch in range(nm_epoch):
        running_loss = 0.0
        for i, data in enumerate(training_loader, 0):
            ids = data['input_ids']
            attention_mask = data['attention_mask']
            token_type_ids = data['token_type_ids']
            labels = data['labels']

            # initialize the optimizer
            optimizer.zero_grad()
            #forward inputs
            outputs = model.forward(ids, attention_mask, token_type_ids)
            # define the loss
            loss = loss_fn(outputs, labels)
            # backpropagate
            loss.backward()
            print("Capturing:", torch.cuda.is_current_stream_capturing())
            optimizer.step()
            # add the loss to the running loss
            running_loss+=loss.item()

            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
    
    try:
        torch.save(model.state_dict(), 'fine_tuned_bert.pt')
        print('Model has been saved !')
    except:
        print('The model has already been saved!')

True

In [59]:

df = pd.read_csv('../data/hate_speech_preprocessed.csv')
df = preprocessing(df)
trainloader, validloader, testloader = dataloader(df, val_frac=0.2, test_frac=0.3, batch_size=8)


loading file vocab.txt from cache at C:\Users\mariu/.cache\huggingface\hub\models--bert-base-uncased\snapshots\0a6aa9128b6194f4f3c4db429b6cb4891cdb421b\vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\mariu/.cache\huggingface\hub\models--bert-base-uncased\snapshots\0a6aa9128b6194f4f3c4db429b6cb4891cdb421b\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\mariu/.cache\huggingface\hub\models--bert-base-uncased\snapshots\0a6aa9128b6194f4f3c4db429b6cb4891cdb421b\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_

In [76]:
m = train(2,trainloader )

loading configuration file config.json from cache at C:\Users\mariu/.cache\huggingface\hub\models--bert-base-uncased\snapshots\0a6aa9128b6194f4f3c4db429b6cb4891cdb421b\config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at C:\Users\mariu/.cache\huggingface\hub\models--bert-base-uncased\snapshots\0a6aa9128b6194f4f3c4db429b6cb4891cdb421b\pytorch_model.bin
Some weights 

AttributeError: 'generator' object has no attribute 'cuda'

In [85]:

optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-5, capturable=False)

model_1 = BERTForFineTuning()
model_1.to('cpu')

def train(epoch):
    model_1.train()
    for _,data in enumerate(trainloader, 0):
        ids = data['input_ids'].to('cpu', dtype = torch.long)
        mask = data['attention_mask'].to('cpu', dtype = torch.long)
        token_type_ids = data['token_type_ids'].to('cpu', dtype = torch.long)
        targets = data['labels'].to('cpu', dtype = torch.float)

        outputs = model_1(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

for epoch in range(2):
    train(epoch)


loading configuration file config.json from cache at C:\Users\mariu/.cache\huggingface\hub\models--bert-base-uncased\snapshots\0a6aa9128b6194f4f3c4db429b6cb4891cdb421b\config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at C:\Users\mariu/.cache\huggingface\hub\models--bert-base-uncased\snapshots\0a6aa9128b6194f4f3c4db429b6cb4891cdb421b\pytorch_model.bin
Some weights 

8
8
Epoch: 0, Loss:  0.7344102263450623


RuntimeError: Attempting CUDA graph capture of step() for an instance of Adam but this instance was constructed with capturable=False.